In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
%matplotlib inline

In [2]:
# read in data
with open('ssdata/ord.5.tab') as f:
    raw_data = [r.replace('\n', '').split('\t') for r in f.readlines()]

df = pd.DataFrame(raw_data[1:], columns=raw_data[0])

In [9]:
# columns --- does the last one (" ") mean unknown?
print raw_data[0]

['pos', 'A-2', 'C-2', 'D-2', 'E-2', 'F-2', 'G-2', 'H-2', 'I-2', 'K-2', 'L-2', 'M-2', 'N-2', 'P-2', 'Q-2', 'R-2', 'S-2', 'T-2', 'V-2', 'W-2', 'Y-2', '--2', 'A-1', 'C-1', 'D-1', 'E-1', 'F-1', 'G-1', 'H-1', 'I-1', 'K-1', 'L-1', 'M-1', 'N-1', 'P-1', 'Q-1', 'R-1', 'S-1', 'T-1', 'V-1', 'W-1', 'Y-1', '--1', 'A0', 'C0', 'D0', 'E0', 'F0', 'G0', 'H0', 'I0', 'K0', 'L0', 'M0', 'N0', 'P0', 'Q0', 'R0', 'S0', 'T0', 'V0', 'W0', 'Y0', '-0', 'A1', 'C1', 'D1', 'E1', 'F1', 'G1', 'H1', 'I1', 'K1', 'L1', 'M1', 'N1', 'P1', 'Q1', 'R1', 'S1', 'T1', 'V1', 'W1', 'Y1', '-1', 'A2', 'C2', 'D2', 'E2', 'F2', 'G2', 'H2', 'I2', 'K2', 'L2', 'M2', 'N2', 'P2', 'Q2', 'R2', 'S2', 'T2', 'V2', 'W2', 'Y2', '-2', 'H', 'E', 'T', 'S', 'B', ' ']


In [10]:
# update the last column name
col_names = list(df.columns)
col_names[-1] = 'U'
df.columns = col_names

In [11]:
df.head()

,pos,A-2,C-2,D-2,E-2,F-2,G-2,H-2,I-2,K-2,...,V2,W2,Y2,-2,H,E,T,S,B,U
0,2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
1,3,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
2,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
3,5,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
4,6,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0


In [12]:
df.shape

(134814, 112)

In [13]:
data = df.copy().drop(['pos', 'H', 'E', 'T', 'S', 'B', 'U'], axis=1)
labels = df.copy()[['H', 'E', 'T', 'S', 'B', 'U']]

In [14]:
# get the labels in the right format
encoding = dict(zip(labels, range(len(labels))))
encoding

{'B': 4, 'E': 1, 'H': 0, 'S': 3, 'T': 2, 'U': 5}

In [15]:
# something like this
np.argmax(list(labels.ix[0]))

5

In [16]:
# clean this up with an apply function
new_labels = np.zeros(len(labels))

for i in xrange(len(labels)):
    new_labels[i] = np.argmax(list(labels.ix[i]))

In [19]:
data.to_csv('5_formatted.csv')

In [22]:
true_lables = pd.DataFrame(new_labels)

In [23]:
true_lables.to_csv('5_formatted_labels.csv')

### Testing RF Classification

In [3]:
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, \
                             AdaBoostClassifier, ExtraTreesClassifier
from sklearn.cross_validation import cross_val_score

In [24]:
# check vs a few different RF classifiers
print cross_val_score(RandomForestClassifier(n_estimators=100), data, y=new_labels, cv=3, n_jobs=-1).mean()
print cross_val_score(ExtraTreesClassifier(n_estimators=100), data, y=new_labels, cv=3, n_jobs=-1).mean()
# print cross_val_score(AdaBoostClassifier(n_estimators=100), data, y=new_labels, cv=5).mean()
# print cross_val_score(GradientBoostingClassifier(n_estimators=100), data, y=new_labels, cv=5).mean()

0.634474288705
0.627865120093


In [ ]:
# trying to compare forest size to 
scores = []
forest_size = xrange(100, 10000, 500)

for size in forest_size:
    score = cross_val_score(RandomForestClassifier(n_estimators=size), 
                            data, y=new_labels, cv=5).mean()
    scores.append(score)

plt.plot(forest_size, scores);